<h1>Tensorflow Datasets und Pipeline</h1>




Siehe: https://www.tensorflow.org/guide/data [Letzter Zugriff: 19.06.2024]

In [ ]:
# // Content Coming

Mit den Datasets von Tensorflow sind einige Dinge möglich, hier werden die Einzelheiten gezeigt.

In [3]:
import tensorflow as tf
import numpy as np

Die Dataset Struktur von Tensorflow bietet viele Möglichkeiten für ETL. 

In [28]:
# Dataset:
data = np.array(range(20))
data

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [29]:
# Füge Dataset negative  und große Werte hinzu. 
data = np.append(data, [-10, -50, 200, 400])
data

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19, -10, -50, 200, 400])

In [30]:
# TF Dataset.:
tf_dataset = tf.data.Dataset.from_tensor_slices(data)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

Durch dieses Dataset kann jetzt iteriert werden. <br>
Um wieder eine Zahl als Numpy zu bekommen, wird die Methode .numpy() angewendet.

Durch das einfache Iterieren kann man sich schnell die Daten anschauen.

In [23]:
for i in tf_dataset:
    #print(i)  # Ausgabe: tf.Tensor(0, shape=(), dtype=int32), ...
    print(i.numpy())  # Ausgabe: 0, ...

    # Oder direkt als Numpy mit: tf_dataset.as_numpy_iterator()

    # Oder nehme nur die ersten 5  mit tf_dataset.take(5):
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
-10
-50
-10
-50
200
400


Ein Dataset kann durch ein Lambda oder eine separate Funktion gefiltert werden. 

In [73]:
def print_dataset(dataset):
    for i in dataset.as_numpy_iterator():
        print(i)

def do_op(x):
    return x*3

In [43]:
# Lambda:
tf_dataset = tf_dataset.filter(lambda x: ( (x>0) & (x<70) ))

In [44]:
print_dataset(tf_dataset)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [76]:
# Wende weitere Operation an.
tf_dataset  = tf_dataset.filter(lambda x:  x>9)
tf_dataset2 = tf_dataset.map(lambda x: x*2)
tf_dataset3 = tf_dataset.map(do_op)  # Mit Funktion. 

In [72]:
print_dataset(tf_dataset2)

20
22
24
26
28
30
32
34
36
38


Die Daten können auch gemischt werden.

In [ ]:
tf_dataset2.shuffle